# Irony Detection in English 

#### The goal of this project consists in training a model capable of performing the task of classificating a tweet in English as ironic or non ironic.

Using different classification techniques we intend to find the one that performs better using as training data a data set with around 4000 tweets of different types.

This project was made by 3 students as part of the Artificial Intelligence course at FEUP. You are free to use the code for any purpose, but beware that this is just an academia project in an introductory course to Artificial Intelligence.

Libraries Used: 
* Pandas
* Numpy
* Scikit-learn
* Matplotlib

### Datasets:

The data sets for this project are not owned by our group, they are supplied by SemEval competition and are free to use for academic purposes only outside of the competition.

There are two data sets available, one marked for train and the other for testing. Originally they were in .txt format, but in order to simplify the work of the pandas library, we manually converted those files to .csv

**SemEval organization themselves already provide a division of the dataset in two subsets, a test dataset and a train dataset.**

This division is useful for the final evaluation of our work. Since the competition was in 2018, the ranking of the participants for that dataset is already public. This way we are able to keep track of which place we could score in that professional competition



![title](../assets/ranking.png)
##### Fig1: Ranking of the SemEval competition

#### Problem with word delimeter

Pandas is a library perfect for the job of reading data from a file, since it setup automaticaly the capability of seeing that data in a formatted organized way.

Pandas has specific functions to read from different formats. 4000 tweets in JSON would turn the data file really dense in information.
Since we are dealing with just 2 attributes, it is pointless to make our life harder in this step. We chose .csv format to represent the data

CSV files dealing with text rise other problem, the fact that people use comma in their texts, pandas. So we introduce other delimiter that exists in Portuguese, but doesn't exist in english "_ç", so we are able to do the task

In [32]:
from IPython.display import display, HTML
import pandas
import numpy

train_data=pandas.read_csv('data/train.csv',  delimiter = '\t', quoting = 3, encoding="utf-8")
test_data=pandas.read_csv('data/test-labeled.csv', delimiter = '\t', quoting = 3, encoding="utf-8")

In [33]:
train_data.head()

,label,tweet
0,1,Sweet United Nations video. Just in time for C...
1,1,@mrdahl87 We are rumored to have talked to Erv...
2,1,Hey there! Nice to see you Minnesota/ND Winter...
3,0,3 episodes left I'm dying over here
4,1,"""I can't breathe!"" was chosen as the most nota..."


In [34]:
test_data.head()

,label,tweet
0,0,@Callisto1947 Can U Help?||More conservatives ...
1,1,"Just walked in to #Starbucks and asked for a ""..."
2,0,#NOT GONNA WIN http://t.co/Mc9ebqjAqj
3,0,@mickymantell He is exactly that sort of perso...
4,1,So much #sarcasm at work mate 10/10 #boring 10...


### Analising the data

With both of our data sets loaded, we can use **Matplotlib** to see how our data is distributed.

In [20]:
import matplotlib.pyplot as plt
%matplotlib inline
train_data_ntrue=train_data['label'].sum()
train_data_nfalse=train_data['label'].count()-train_data_ntrue
values=[train_data_ntrue,train_data_nfalse]
names=['Ironic','Not ironic']
plt.bar(names,values)
plt.suptitle('Train data set - distribution')
plt.show()
print('Ironic tweets - ',train_data_ntrue)
print('Non Ironic tweets - ',train_data_nfalse)
print('Percentage of ironic tweets - ',train_data_ntrue*1.0/train_data['label'].count()*100.0)

KeyError: 'label'

In [15]:
test_data_ntrue=test_data['label'].sum()
test_data_nfalse=test_data['label'].count()-test_data_ntrue
values=[test_data_ntrue,test_data_nfalse]
names=['Ironic','Non Ironic']
plt.suptitle('Test data set - distribution')
plt.bar(names,values)
plt.show()
print('Ironic tweets - ',test_data_ntrue)
print('Non Ironic tweets - ',test_data_nfalse)
print('Percentage of ironic tweets - ',test_data_ntrue*1.0/test_data['label'].count()*100.0)

KeyError: 'label'


While the training data set has a marginally small difference between the number of ironic and non ironic tweets, the same can't be said for the test data set.
In our second graph, its visually palpable that the number of non ironic tweets is significantly larger.

### Bag of words

Now that we have the data in a pandas data set we need to proceed our path of finding an accurate model to identify ironic tweets. For that we need to parse our tweets to a **bag of words** model format.

A bag of words is a model which tweet will see their words tokenized and counted by a specified formula. Where we have 2 essential paths that we can follow:

* Use a simple counting of words.
* Use the TF-IDF measure

##### The process of getting as input the raw text and output a bag of words is called **Vectorization**



## Data Preprocessing

# TODO -> Remove non english words? / Tokenize emojis / Lemmatization 

In [7]:
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

corpus = []
stemmer = PorterStemmer()

for tweet in train_data['tweet']:
    # Remove urls
    tweet = re.sub(r"http\S+|www\S+|https\S+", '', tweet)
    # get tweet and remove non alpha chars (except spaces)
    tweet = re.sub('[^a-zA-Z +]', '', tweet)
    # Word tokenization and lowercasing
    tweet = tweet.lower().split()
    #Stemming and stop word removal
    tweet = ' '.join([stemmer.stem(w) for w in tweet if not w in set(stopwords.words('english'))])
    corpus.append(tweet)
print(corpus)

LookupError: 
**********************************************************************
  Resource [93mstopwords[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('stopwords')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mcorpora/stopwords[0m

  Searched in:
    - '/home/ruben/nltk_data'
    - '/home/ruben/anaconda3/nltk_data'
    - '/home/ruben/anaconda3/share/nltk_data'
    - '/home/ruben/anaconda3/lib/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
**********************************************************************


## Initial analysis

In order to have a objective evaluation criteria to our final answer to this problem, it is important to make a fast initial analysis to the data without any kind of pre processing to the data, using the simplest model possible a naive bayes analyses.

#### Simple Word Counter Vetorization
#### Train Data

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

count_vectorizer = CountVectorizer(analyzer='word', stop_words='english', lowercase=False)

X_train=count_vectorizer.fit_transform(corpus).toarray()
y_train=train_data['label']

#### Test Data

In [ ]:
# Here we just Transform. Because the vocabulary is the one from the test set
X_test=count_vectorizer.transform(test_data['tweet']).toarray()
y_test=test_data['label']

### TF-IDF Word Counter Vetorization

In [ ]:
classifier=GaussianNB()

classifier.fit(X_train,y_train)

y_predicted=classifier.predict(X_test)

print(classification_report(y_test,y_predicted))
print(confusion_matrix(y_test, y_predicted))

In [ ]:
# Decision Tree

from sklearn.tree import DecisionTreeClassifier

classifier = DecisionTreeClassifier()
classifier.fit(X_train, y_train)
y_predicted = classifier.predict(X_test)

print(classification_report(y_test,y_predicted))
print(confusion_matrix(y_test, y_predicted))

In [ ]:
# Neural Network

from sklearn.neural_network import MLPClassifier

classifier = MLPClassifier()
classifier.fit(X_train, y_train)
y_predicted = classifier.predict(X_test)

print(classification_report(y_test,y_predicted))
print(confusion_matrix(y_test, y_predicted))

In [ ]:
# KNN

from sklearn.neighbors import KNeighborsClassifier

classifier = KNeighborsClassifier()
classifier.fit(X_train, y_train)
y_predicted = classifier.predict(X_test)

print(classification_report(y_test,y_predicted))
print(confusion_matrix(y_test, y_predicted))

In [ ]:
# SVM

from sklearn.svm import SVC

classifier = SVC()
classifier.fit(X_train, y_train)
y_predicted=classifier.predict(X_test)

print(classification_report(y_test,y_predicted))
print(confusion_matrix(y_test, y_predicted))

#### Train Data

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

tf_idf_vectorizer=TfidfVectorizer(analyzer='word', stop_words='english', lowercase=False)

X_train=tf_idf_vectorizer.fit_transform(train_data['tweet']).toarray()
y_train=train_data['label']

#### Test Data

In [ ]:
X_test=tf_idf_vectorizer.transform(test_data['tweet']).toarray()
y_test=test_data['label']

In [ ]:
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import accuracy_score

classificator=GaussianNB()

classificator.fit(X_train,y_train)
#Predicition
y_predicted=classificator.predict(X_test)



#Metrics Raw
accuracy_value=accuracy_score(y_test,y_predicted)
precision_value=precision_score(y_test,y_predicted,average='weighted')
f1_value=f1_score(y_test,y_predicted,average='weighted')

print(classification_report(y_test,y_predicted))
print("Confusion Matrix")
print(confusion_matrix(y_test, y_predicted))



initial_conclusion=pandas.DataFrame(
    {
    'Our Results':[accuracy_value,precision_value,f1_value],
    'Winners Result':[0.7347,0.6304,0.8006]
    } , index=['Accuracy','Precision','F1-Value'])

### Initial Data Analys

The raw analysis of the perfomance of the simple model is pretty bellow the best scores of the competition:

In [ ]:
initial_conclusion

#### The differences are relevant. We need to introduce data pre processing to our project in order to improve the data

Other element relevant is that using simple word counting or TF-IDF doesn't produce any kind of relevant difference in the results achieved


## Data pre processing process

Without any data preprocessing the vocabulary of the testing set was 14042
Normalization

#### Remove Web Hyperlinks from tweets

In [ ]:
train_data['tweet']=train_data['tweet'].str.replace('http\S+|www.\S+', '', case=False)

#Deletes any kind of empty tweet that was left after removing links
train_data.replace('', numpy.nan, inplace=True)
train_data.dropna(subset=['tweet'],inplace=True)

display(HTML(train_data.to_html()))

#### Normalization of Twitter mentions

In twitter is mandatory to mention the person that we reply with their personal nickname. In this Social Network this mentions are done using "@" followed by a single word that identifies the nickname

In [ ]:
train_data['tweet']=train_data['tweet'].str.replace('@\S+', '<person>', case=False)

display(HTML(train_data.to_html()))

#### Normalization of Uppercase words.

Goal is to reduce the vocabulary to the absolutely necessary

In [ ]:
#Data Refresh
tf_idf_vectorizer=TfidfVectorizer(analyzer='word', stop_words='english', lowercase=True)

X_train=tf_idf_vectorizer.fit_transform(train_data['tweet']).toarray()
y_train=train_data['label']

In [ ]:
print(len(tf_idf_vectorizer.vocabulary_))

In [ ]:
14042

* Fazer aquela cena dos vetores de similitude vai curtir -> Encontrar duplicados
* Remover Numeros
* Fazer Stemming
* Fazer Lemmazation
* Fazer contradictions

In [ ]:

X_test=tf_idf_vectorizer.transform(test_data['tweet']).toarray()
y_test=test_data['label']

classificator.fit(X_train,y_train)
#Predicition
y_predicted=classificator.predict(X_test)

print(classification_report(y_test,y_predicted))
print("Confusion Matrix")
print(confusion_matrix(y_test, y_predicted))

accuracy_value=accuracy_score(y_test,y_predicted)
precision_value=precision_score(y_test,y_predicted,average='weighted')
f1_value=f1_score(y_test,y_predicted,average='weighted')

actual_conclusion=pandas.DataFrame(
    {
    'Our Results':[accuracy_value,precision_value,f1_value],
    'Winners Result':[0.7347,0.6304,0.8006]
    } , index=['Accuracy','Precision','F1-Value'])

In [ ]:
actual_conclusion